In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import pickle

In [2]:
cw_map = {'cancer': 4,
 'cervicalcanc': 6,
 'gardasil': 2,
 'health': 0,
 'hpv': 9,
 'hpvvaccin': 3,
 'learntherisk': 8,
 'studi': 7,
 'vaccin': 5,
 'vaccineswork': 10,
 'vax': 1}

idx2ht = {v: k for k, v in cw_map.items()}

In [3]:
file = "predict_results_1biLSTM"
df = pd.read_csv("predict_res/"+file+".csv")
df.head(5)

,tweet_text,true_label,health,vax,gardasil,hpvvaccin,cancer,vaccin,cervicalcanc,studi,learntherisk,hpv,vaccineswork
0,vaccine cuts teen girls ' hpv rates by two-thi...,health,0.997747,0.001123,6.564490e-07,2.698930e-07,3.316900e-07,0.001125,6.192670e-07,1.152250e-06,4.404900e-07,8.083010e-07,1.055320e-07
1,"rt : . vaxxed update ! gardasil tragedy , lead...",hpv,0.000021,0.000244,5.348350e-01,2.528250e-05,7.599710e-06,0.000062,6.075810e-06,1.809800e-08,7.034950e-06,4.647920e-01,7.421230e-07
2,rt : are you another gardasil pusherman ?,gardasil,0.000124,0.000863,9.986260e-01,5.476730e-05,3.448030e-05,0.000057,3.026150e-05,3.472100e-06,2.307380e-05,1.556260e-04,2.886010e-05
3,hpv vaccination rates highest among hispanic g...,"vax, vaccin",0.000151,0.576814,1.727790e-04,1.593560e-04,2.323190e-05,0.422348,4.732910e-05,4.744580e-05,2.285810e-04,1.357940e-06,6.817710e-06
4,rt : online : integrated genomic and molecular...,cervicalcanc,0.000022,0.000391,9.919180e-06,2.330830e-05,3.723770e-05,0.000001,9.993910e-01,1.153040e-04,9.099290e-07,2.238840e-06,4.609420e-06


In [4]:
#add a columen with a list of combining all the probabilities
fields = list(df)[2:]
l = []
for i, row in df.iterrows():
    l.append(list(row[h] for h in fields))
df['p'] = pd.Series(l).values
df.head(5)

,tweet_text,true_label,health,vax,gardasil,hpvvaccin,cancer,vaccin,cervicalcanc,studi,learntherisk,hpv,vaccineswork,p
0,vaccine cuts teen girls ' hpv rates by two-thi...,health,0.997747,0.001123,6.564490e-07,2.698930e-07,3.316900e-07,0.001125,6.192670e-07,1.152250e-06,4.404900e-07,8.083010e-07,1.055320e-07,"[0.9977469999999999, 0.00112323, 6.56449000000..."
1,"rt : . vaxxed update ! gardasil tragedy , lead...",hpv,0.000021,0.000244,5.348350e-01,2.528250e-05,7.599710e-06,0.000062,6.075810e-06,1.809800e-08,7.034950e-06,4.647920e-01,7.421230e-07,"[2.05549e-05, 0.000243562, 0.5348350000000001,..."
2,rt : are you another gardasil pusherman ?,gardasil,0.000124,0.000863,9.986260e-01,5.476730e-05,3.448030e-05,0.000057,3.026150e-05,3.472100e-06,2.307380e-05,1.556260e-04,2.886010e-05,"[0.00012371799999999998, 0.0008626560000000001..."
3,hpv vaccination rates highest among hispanic g...,"vax, vaccin",0.000151,0.576814,1.727790e-04,1.593560e-04,2.323190e-05,0.422348,4.732910e-05,4.744580e-05,2.285810e-04,1.357940e-06,6.817710e-06,"[0.000150698, 0.576814, 0.00017277900000000002..."
4,rt : online : integrated genomic and molecular...,cervicalcanc,0.000022,0.000391,9.919180e-06,2.330830e-05,3.723770e-05,0.000001,9.993910e-01,1.153040e-04,9.099290e-07,2.238840e-06,4.609420e-06,"[2.24698e-05, 0.00039149800000000005, 9.91918e..."


In [5]:
probs = np.array(df['p'])
probs[:5]

array([list([0.9977469999999999, 0.00112323, 6.564490000000001e-07, 2.69893e-07, 3.3169000000000003e-07, 0.00112508, 6.19267e-07, 1.1522499999999999e-06, 4.4049e-07, 8.08301e-07, 1.0553200000000001e-07]),
       list([2.05549e-05, 0.000243562, 0.5348350000000001, 2.52825e-05, 7.59971e-06, 6.17787e-05, 6.07581e-06, 1.8098e-08, 7.03495e-06, 0.464792, 7.42123e-07]),
       list([0.00012371799999999998, 0.0008626560000000001, 0.998626, 5.476729999999999e-05, 3.44803e-05, 5.70348e-05, 3.02615e-05, 3.4720999999999998e-06, 2.3073800000000003e-05, 0.000155626, 2.8860100000000002e-05]),
       list([0.000150698, 0.576814, 0.00017277900000000002, 0.000159356, 2.32319e-05, 0.42234799999999995, 4.73291e-05, 4.74458e-05, 0.00022858099999999997, 1.3579399999999998e-06, 6.817710000000001e-06]),
       list([2.24698e-05, 0.00039149800000000005, 9.91918e-06, 2.33083e-05, 3.72377e-05, 1.4365100000000002e-06, 0.999391, 0.00011530399999999999, 9.09929e-07, 2.2388400000000003e-06, 4.609419999999999e-06])],

In [6]:
#separate data into groups
def get_groups(data, num_gp):
    groups = {k: [] for k in range(num_gp)}
    for each in data:
        fmax = -1
        tag = -1
        for i, prob in enumerate(each):
            if prob > fmax:
                tag = i
                fmax = prob
        groups[tag].append(each)
    return groups

gps = get_groups(probs, 11)        

In [7]:
#clusters basic information
for k, v in gps.items():
    print("Group {}: {}".format(k, idx2ht[k]))
    print("Total elements: {}".format(len(v)))

Group 0: health
Total elements: 96
Group 1: vax
Total elements: 1418
Group 2: gardasil
Total elements: 301
Group 3: hpvvaccin
Total elements: 58
Group 4: cancer
Total elements: 88
Group 5: vaccin
Total elements: 374
Group 6: cervicalcanc
Total elements: 275
Group 7: studi
Total elements: 1
Group 8: learntherisk
Total elements: 488
Group 9: hpv
Total elements: 57
Group 10: vaccineswork
Total elements: 86


In [8]:
from scipy import stats
def dis(di, dj):
    di = np.asarray(di, dtype = np.float)
    dj = np.asarray(dj, dtype = np.float)
    m = 0.5 * (di + dj)
    k_l_l = stats.entropy(di, m)
    k_l_r = stats.entropy(dj, m)  # k_l_1 = np.sum(np.where(di != 0, di * np.log(di / dj), 0))
    dis = 0.5 * (k_l_l + k_l_r)
    return dis

In [22]:
#intra distance
cluster_num = 11
total_intra = 0
for i in range(cluster_num):
    cluster_sum = 0
    current_cluster = gps[i]
    element_num = len(current_cluster)
    for i in range(element_num):
        for j in range(i+1, element_num):
            cluster_sum += (2*dis(current_cluster[i], current_cluster[j]) / element_num * (element_num - 1))
    total_intra += (1/cluster_num)*cluster_sum

total_intra

29382.837931627433

In [20]:
#inter distance
cluster_num = 11
total_inter = 0

for k1 in range(cluster_num):
    cluster_sum = 0
    for k2 in range(cluster_num):
        if k1 != k2:
            current_cluster1 = gps[k1]
            current_cluster2 = gps[k2]
            for i in range(len(current_cluster1)):
                for j in range(len(current_cluster2)):
                    cluster_sum += (dis(current_cluster1[i], current_cluster2[j]) / len(current_cluster1) * len(current_cluster2))
    total_inter += (1/(cluster_num*(cluster_num - 1))) * cluster_sum

total_inter

126372.58930412814

In [21]:
#H score
H = total_intra / total_inter

H

0.23250958212872197

In [9]:
#on large data set
file = "predict_results_on_no_label_sample1biLSTM"
df1 = pd.read_csv("predict_res/"+file+".csv")
file = "predict_results_1biLSTM"
df = pd.read_csv("predict_res/"+file+".csv")
df2 = pd.concat([df, df1])

fields = list(df2)[2:]
l = []

for i, row in df2.iterrows():
    l.append(list(row[h] for h in fields))
df2['p'] = pd.Series(l).values

probs = np.array(df2['p'])
gps = get_groups(probs, 11)  

In [10]:
for k, v in gps.items():
    print("Group {}: {}".format(k, idx2ht[k]))
    print("Total elements: {}".format(len(v)))

Group 0: health
Total elements: 1890
Group 1: vax
Total elements: 87624
Group 2: gardasil
Total elements: 14226
Group 3: hpvvaccin
Total elements: 61
Group 4: cancer
Total elements: 249
Group 5: vaccin
Total elements: 13228
Group 6: cervicalcanc
Total elements: 40215
Group 7: studi
Total elements: 206
Group 8: learntherisk
Total elements: 1613
Group 9: hpv
Total elements: 200
Group 10: vaccineswork
Total elements: 98


In [11]:
# random select 200 from each group
new_gps = dict()
for k, v in gps.items():
    np.random.shuffle(v)
    new_gps[k] = v[:200]

for k, v in new_gps.items():
    print("Group {}: {}".format(k, idx2ht[k]))
    print("Total elements: {}".format(len(v)))

Group 0: health
Total elements: 200
Group 1: vax
Total elements: 200
Group 2: gardasil
Total elements: 200
Group 3: hpvvaccin
Total elements: 61
Group 4: cancer
Total elements: 200
Group 5: vaccin
Total elements: 200
Group 6: cervicalcanc
Total elements: 200
Group 7: studi
Total elements: 200
Group 8: learntherisk
Total elements: 200
Group 9: hpv
Total elements: 200
Group 10: vaccineswork
Total elements: 98


In [ ]:
%%time
cluster_num = 11
total_inter = 0
total_intra = 0

for i in range(cluster_num):
    cluster_sum = 0
    current_cluster = gps[i]
    element_num = len(current_cluster)
    for i in range(element_num):
        for j in range(i+1, element_num):
            cluster_sum += (2*dis(current_cluster[i], current_cluster[j]) / element_num * (element_num - 1))
    total_intra += (1/cluster_num)*cluster_sum

for k1 in range(cluster_num):
    cluster_sum = 0
    for k2 in range(cluster_num):
        if k1 != k2:
            current_cluster1 = gps[k1]
            current_cluster2 = gps[k2]
            for i in range(len(current_cluster1)):
                for j in range(len(current_cluster2)):
                    cluster_sum += (dis(current_cluster1[i], current_cluster2[j]) / len(current_cluster1) * len(current_cluster2))
    total_inter += (1/(cluster_num*(cluster_num - 1))) * cluster_sum

H = total_intra / total_inter
H